In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [ ]:
import torch
import pandas as pd
import numpy as np

In [ ]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df = pd.read_csv('/content/Complaint data annotation (explain)_updated - cd (1).csv')

In [ ]:
df.keys()

Index(['id', 'tweet', 'label', 'domain', 'sentiment', 'emotion', 'Severity',
       'Explain'],
      dtype='object')

In [ ]:
df.head()

,id,tweet,label,domain,sentiment,emotion,Severity,Explain
0,1,@FC_HELP can I return online purchases to a Ho...,0,apparel,Neutral,other,0,can I return online purchases to a House of Fr...
1,2,@FC_Help Hi - I'm writing a piece for MSN Him ...,0,apparel,Positive,other,0,Hi - I'm writing a piece for MSN Him and wonde...
2,3,@FC_Help i need to check my order,0,apparel,Neutral,other,0,i need to check my order
3,4,@FC_Help I need to get in contact with someone...,1,apparel,Neutral,other,1,I need to get in contact with someone regardin...
4,5,@FC_Help How can I get a hold of you so we can...,0,apparel,Negative,other,0,How can I get a hold of you so we can discuss ...


In [ ]:
domain_dict = {
    "other" : 0,
    "services" : 1,
    "random_reply" : 2,
    "software" : 3,
    "retail" : 4,
    "random_tweet" : 5,
    "transport" : 6,
    "cars" : 7,
    "food" : 8,
    "apparel" : 9,
    "electronics" : 10
}

# note to take string lower
senti_dict = {
    'negative' : 0,
    'positive' : 1,
    'neutral' : 2
}

emo_dict = {
    'sadness' : 0, 
    'joy' : 1, 
    'other' : 2, 
    'anger' : 3, 
    'disgust' : 4, 
    'surprise' : 5, 
    'fear' : 6
}

In [ ]:
label = []
domain = []
emotion = []
sentiment = []
severity = []
explain = []

for i in range(len(df)):
  if (pd.isna(df['label'][i]) or pd.isna(df['domain'][i]) or pd.isna(df['emotion'][i]) or pd.isna(df['sentiment'][i]) or pd.isna(df['Severity'][i]) or pd.isna(df['Explain'][i])):
    continue
  label.append(df['label'][i])
  domain.append(domain_dict[df['domain'][i]])
  emotion.append(emo_dict[df['emotion'][i]])
  sentiment.append(senti_dict[(df['sentiment'][i]).lower()])
  severity.append(df['Severity'][i])
  explain.append(df['Explain'][i])

In [ ]:
from sklearn.model_selection import train_test_split

label_train, label_test, domain_train, domain_test, emotion_train, emotion_test, sentiment_train, sentiment_test, severity_train, severity_test, explain_train, explain_test = train_test_split(label, domain, emotion, sentiment, severity, explain, test_size = 0.2, random_state = 42, shuffle = True)
label_train, label_val, domain_train, domain_val, emotion_train, emotion_val, sentiment_train, sentiment_val, severity_train, severity_val, explain_train, explain_val = train_test_split(label_train, domain_train, emotion_train, sentiment_train, severity_train, explain_train, test_size = 0.2, random_state = 42, shuffle = True)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_data(text):
  g_input_ids = []
  g_attention_mask = []

  for line in text:
    inputs = tokenizer(line, return_tensors="pt")

    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    size = input_ids.shape[1]

    if size < 60:
      input_ids = torch.cat((input_ids, torch.zeros(1, 60-size)), dim = 1)
      attention_mask = torch.cat((attention_mask, torch.zeros(1, 60-size)), dim = 1)
    else:
      input_ids = input_ids[:, :60]
      attention_mask = attention_mask[:, :60]
    
    # Why we use np.array()
    g_input_ids.append(np.array(input_ids.reshape(-1)))
    g_attention_mask.append(np.array(attention_mask.reshape(-1)))

  g_input_ids = torch.tensor(g_input_ids)
  g_attention_mask = torch.tensor(g_attention_mask)

  return g_input_ids, g_attention_mask

In [ ]:
input_ids_train, attention_mask_train = preprocess_data(explain_train)
input_ids_val, attention_mask_val = preprocess_data(explain_val)
input_ids_test, attention_mask_test = preprocess_data(explain_test)

In [ ]:
class Complaint_dataset():
  def __init__(self, input_ids, attention_mask, label, domain, emotion, sentiment, severity):
    self.input_ids = input_ids
    self.attention_mask = attention_mask
    self.label = label
    self.domain = domain
    self.emotion = emotion
    self.sentiment = sentiment
    self.severity = severity
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    
    sample = {
        "input_ids" : torch.tensor(self.input_ids[idx]).long().to(device),
        "attention_mask" : torch.tensor(self.attention_mask[idx]).long().to(device),
        "label" : torch.tensor(self.label[idx]).long().to(device),
        "domain" : torch.tensor(self.domain[idx]).long().to(device),
        "emotion" : torch.tensor(self.emotion[idx]).long().to(device),
        "sentiment" : torch.tensor(self.sentiment[idx]).long().to(device),
        "severity" : torch.tensor(self.severity[idx]).long().to(device)
    }

    return sample

In [ ]:
from torch.utils.data import DataLoader

complaint_train = Complaint_dataset(input_ids_train, attention_mask_train, label_train, domain_train, emotion_train, sentiment_train, severity_train)
train_dataloader = DataLoader(complaint_train, batch_size = 32, shuffle = False)

complaint_val = Complaint_dataset(input_ids_val, attention_mask_val, label_val, domain_val, emotion_val, sentiment_val, severity_val)
val_dataloader = DataLoader(complaint_val, batch_size = 32, shuffle = False)

complaint_test = Complaint_dataset(input_ids_test, attention_mask_test, label_test, domain_test, emotion_test, sentiment_test, severity_test)
test_dataloader = DataLoader(complaint_test, batch_size = 32, shuffle = False)

In [ ]:
import torch.nn as nn
from transformers import BertModel

class Emotion(nn.Module):
  def __init__(self, classes):
    super(Emotion, self).__init__()
    self.emo_a = nn.Linear(768, 512).to(device)
    self.emo_b = nn.Linear(512, 256).to(device)
    self.emo_c = nn.Linear(256, classes).to(device)
    self.relu = nn.ReLU()
    self.softmax = nn.Softmax(dim = 1).to(device)
  def forward(self, bert_embed):
    emo_a = self.relu(self.emo_a(bert_embed))
    emo_b = self.relu(self.emo_b(emo_a))
    emo_out = self.softmax(self.emo_c(emo_b))
    return emo_a, emo_b, emo_out

class Complaint(nn.Module):
  def __init__(self):
    super(Complaint, self).__init__()
    self.bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)
    self.emo = Emotion(classes = 7)

    self.emoti.load_state_dict(torch.load("bert_emo.pt"))

    self.central_b = nn.Linear(512, 256).to(device)
    self.central_c = nn.Linear(256, 128).to(device)
    self.central_comp = nn.Linear(128, 2).to(device)
    self.central_sev = nn.Linear(128, 5).to(device)
    self.relu = nn.ReLU()
    self.softmax = nn.Softmax(dim = 1).to(device)
  def forward(self, input_ids, attention_mask):
    bert_embed = self.bert_model(input_ids = input_ids.to(device), attention_mask = attention_mask.to(device)).pooler_output.to(device)

    for p in self.emo.parameters():
      p.require_grads = False

    emo_a, emo_b, emo_out = self.emo(bert_embed)

    batch_size = emo_a.shape[0]

    #parameters
    self.a = nn.ParameterList([nn.Parameter(torch.rand(1).to(device)) for i in range(4)])
    self.b = nn.ParameterList([nn.Parameter(torch.rand(1).to(device)) for i in range(4)])
    self.c = nn.ParameterList([nn.Parameter(torch.rand(1).to(device)) for i in range(4)])

    central_a = torch.zeros(batch_size, 512).to(device)
    wsum_a = self.a[0].expand_as(central_a)*central_a + self.b[0].expand_as(emo_a)*emo_a

    central_b = self.relu(self.central_b(wsum_a)).to(device)
    wsum_b = self.a[1].expand_as(central_b)*central_b + self.b[1].expand_as(emo_b)*emo_b

    central_c = self.relu(self.central_c(wsum_b)).to(device)

    # now the emo and senti are done with the output so they have their classes size
    # we need to concat them
    emo_out = torch.cat((emo_out, torch.zeros(batch_size, 128-emo_out.shape[1]).to(device)), dim = 1).to(device)
    # since we have to concatenate horizontally on all the 32 samples in the batch

    wsum_out = self.a[2].expand_as(central_c)*central_c + self.b[2].expand_as(emo_out)*emo_out

    central_comp = self.softmax(self.central_comp(wsum_out)).to(device)
    sev_out = self.softmax(self.central_sev(wsum_out)).to(device)

    return central_comp, emo_out, sev_out

    # need to add alphas and then do the data processing

In [ ]:
BERT_MODEL = BertModel.from_pretrained('bert-base-uncased').to(device)

In [ ]:
emotion_model = Emotion(classes = 7)
emotion_model = emotion_model.to(device)
emotion_model.train()

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(emotion_model.parameters(), lr = 1e-5, weight_decay = 0)
epochs = 20
min_val_loss = 10

emotion_model.train()

for i in range(epochs):

  for data in train_dataloader:
    emotion_model.zero_grad()
    emo_train = data["emotion"].to(device)
    EMBED = BERT_MODEL(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device)).pooler_output.to(device)
    _1, _2, emo_out = emotion_model.forward(EMBED)

    loss_train = loss_func(emo_out, emo_train)
    loss_train.backward()
    optimizer.step()

    emotion_model.eval()

    with torch.no_grad():
      total_loss_val = 0
      for data in val_dataloader:
        emo_val = data["emotion"].to(device)
        EMBED = BERT_MODEL(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device)).pooler_output.to(device)
        _1, _2, emo_val_out = emotion_model(EMBED)

        val_loss = loss_func(emo_val_out, emo_val)

        total_val = emo_val.size(0)
        total_loss_val += val_loss.item()

      val_loss = total_loss_val / total_val

    if ((min_val_loss-val_loss) > 1e-4):
      min_val_loss = val_loss
      torch.save(emotion_model.state_dict(), "bert_emo.pt")

  print(f"Epoch : {i+1}")
  print(f"Validation loss : {val_loss}")

  emotion_model.load_state_dict(torch.load("bert_emo.pt"))

In [ ]:
complaint_model = Complaint()
complaint_model = complaint_model.to(device)
complaint_model.train()

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(complaint_model.parameters(), lr = 1e-5, weight_decay = 0)
epochs = 20
min_val_loss = 10

complaint_model.train()

for i in range(epochs):

  for data in train_dataloader:
    complaint_model.zero_grad()
    comp_train = data["label"].to(device)
    sev_train = data["severity"].to(device)
    comp_out, emo_out, sev_out = complaint_model.forward(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device))

    loss_train = loss_func(comp_out, comp_train) + loss_func(sev_out, sev_train)
    loss_train.backward()
    optimizer.step()

    complaint_model.eval()

    with torch.no_grad():
      total_loss_val = 0
      for data in val_dataloader:
        comp_val = data["label"].to(device)
        sev_val = data["severity"].to(device)
        comp_val_out, emo_val_out, sev_val_out = complaint_model(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device))

        val_loss = loss_func(comp_val_out, comp_val) + loss_func(sev_val_out, sev_val)

        total_val = comp_val.size(0)
        total_loss_val += val_loss.item()

      val_loss = total_loss_val / total_val

    if ((min_val_loss-val_loss) > 1e-4):
      min_val_loss = val_loss
      torch.save(complaint_model.state_dict(), "bert_model.pt")

  print(f"Epoch : {i+1}")
  print(f"Validation loss : {val_loss}")

  complaint_model.load_state_dict(torch.load("bert_model.pt"))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-31-aa0febf124e5>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.cl

Epoch : 1
Validation loss : 5.157024383544922
Epoch : 2
Validation loss : 5.105437994003296
Epoch : 3
Validation loss : 4.942259609699249
Epoch : 4
Validation loss : 4.738932743668556
Epoch : 5
Validation loss : 4.514396846294403
Epoch : 6
Validation loss : 4.340899407863617
Epoch : 7
Validation loss : 4.294280827045441
Epoch : 8
Validation loss : 4.24046166241169
Epoch : 9
Validation loss : 4.134337827563286
Epoch : 10
Validation loss : 4.162714719772339
Epoch : 11
Validation loss : 3.981364443898201
Epoch : 12
Validation loss : 4.356122747063637
Epoch : 13
Validation loss : 4.17960786819458
Epoch : 14
Validation loss : 4.0929644256830215
Epoch : 15
Validation loss : 4.337694734334946
Epoch : 16
Validation loss : 4.193168759346008
Epoch : 17
Validation loss : 4.2962735295295715
Epoch : 18
Validation loss : 4.253510341048241
Epoch : 19
Validation loss : 4.161923706531525
Epoch : 20
Validation loss : 4.238618925213814


In [ ]:
from sklearn.metrics import *

In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryF1Score, BinaryRecall
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassF1Score, MulticlassRecall

comp_accuracy = BinaryAccuracy()
comp_precision = BinaryPrecision()
comp_f1 = BinaryF1Score()
comp_recall = BinaryRecall() 

sev_accuracy = MulticlassAccuracy(num_classes=5)
sev_precision = MulticlassPrecision(num_classes=5)
sev_f1 = MulticlassF1Score(num_classes=5)
sev_recall = MulticlassRecall(num_classes=5)

emo_accuracy = MulticlassAccuracy(num_classes=7)
emo_precision = MulticlassPrecision(num_classes=7)
emo_f1 = MulticlassF1Score(num_classes=7)
emo_recall = MulticlassRecall(num_classes=7)

with torch.no_grad():
    for data in test_dataloader:
      lab_comp = data["label"]
      lab_emo = data["emotion"]
      lab_sev = data["severity"]

      comp_out, emo_out, sev_out = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])

      _, pred_comp = torch.max(comp_out.data,1)
      _, pred_emo = torch.max(emo_out.data,1)
      _, pred_sev = torch.max(sev_out.data,1)

      comp_accuracy.update(pred_comp.cpu(), lab_comp.cpu())
      comp_precision.update(pred_comp.cpu(), lab_comp.cpu()) 
      comp_f1.update(pred_comp.cpu(), lab_comp.cpu())
      comp_recall.update(pred_comp.cpu(), lab_comp.cpu())

      sev_accuracy.update(pred_sev.cpu(), lab_sev.cpu())
      sev_precision.update(pred_sev.cpu(), lab_sev.cpu())
      sev_f1.update(pred_sev.cpu(), lab_sev.cpu())
      sev_recall.update(pred_sev.cpu(), lab_sev.cpu())

      emo_accuracy.update(pred_emo.cpu(), lab_emo.cpu())
      emo_precision.update(pred_emo.cpu(), lab_emo.cpu())
      emo_f1.update(pred_emo.cpu(), lab_emo.cpu())
      emo_recall.update(pred_emo.cpu(), lab_emo.cpu())
      emo_test_accuracy = emo_accuracy.compute()

    comp_test_accuracy = comp_accuracy.compute()
    comp_test_precision = comp_precision.compute()
    comp_test_f1 = comp_f1.compute()
    comp_test_recall = comp_recall.compute()

    sev_test_accuracy = sev_accuracy.compute()
    sev_test_precision = sev_precision.compute()
    sev_test_f1 = sev_f1.compute()
    sev_test_recall = sev_recall.compute()

    emo_test_accuracy = emo_accuracy.compute()
    emo_test_precision = emo_precision.compute()
    emo_test_f1 = emo_f1.compute()
    emo_test_recall = emo_recall.compute()

    print(f"comp_test_accuracy : {comp_test_accuracy}")
    print(f"comp_test_precision : {comp_test_precision}")
    print(f"comp_test_f1 : {comp_test_f1}")
    print(f"comp_test_recall : {comp_test_recall}") 

    print(f"sev_test_accuracy : {sev_test_accuracy}")
    print(f"sev_test_precision : {sev_test_precision}")
    print(f"sev_test_f1 : {sev_test_f1}")
    print(f"sev_test_recall : {sev_test_recall}") 

    print(f"emo_test_accuracy : {emo_test_accuracy}")
    print(f"emo_test_precision : {emo_test_precision}")
    print(f"emo_test_f1 : {emo_test_f1}")
    print(f"emo_test_recall : {emo_test_recall}")

<ipython-input-31-aa0febf124e5>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids" : torch.tensor(self.input_ids[idx]).long().to(device),
<ipython-input-31-aa0febf124e5>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask" : torch.tensor(self.attention_mask[idx]).long().to(device),


comp_test_accuracy : 0.8623188138008118
comp_test_precision : 0.8611111044883728
comp_test_f1 : 0.7965738773345947
comp_test_recall : 0.7410358786582947
sev_test_accuracy : 0.33256080746650696
sev_test_precision : 0.28933557868003845
sev_test_f1 : 0.3090527653694153
sev_test_recall : 0.33256080746650696
emo_test_accuracy : 0.1428571492433548
emo_test_precision : 0.0003001200675498694
emo_test_f1 : 0.0005989816854707897
emo_test_recall : 0.1428571492433548
